In [1]:
!pip install transformers

import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

import pandas as pd

import re

from transformers import pipeline

from google.colab import drive

import pandas as pd

from pandas.io.formats.info import pprint_thing

from google.colab import drive

drive.mount('/content/drive')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:

def assess_sentiment_model(text, model_name):
    sentiment_analyzer = pipeline("sentiment-analysis", model=model_name)
    result = sentiment_analyzer(text)
    sentiment = result[0]['label']
    score = result[0]['score']
    return sentiment, score


def get_bbc_content(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser") 
    content = soup.find("div", attrs={"class": "gel-layout__item gel-2/3@l"})
    
    if content is None:
        content = soup.find("td", attrs={"class": "storybody"})
        if content is None:
            return None

    main_body = content.find_all('p')
    text = ""
    for i in main_body:
        text += i.text
    clean_text = text.replace('\n', ' ').replace('\xa0', '')
    return clean_text

def assess_sentiment(df):
    sentiment_models = [
        ("bert-base-uncased", "sentiment_bert"),
        ("roberta-base", "sentiment_roberta"),
        ("xlnet-base-cased", "sentiment_xlnet"),
        ("distilbert-base-uncased", "sentiment_distilbert")
    ]

    for _, sentiment_column in sentiment_models:
        df[sentiment_column + "_average_score"] = None

    for index, row in df.iterrows():
        content = get_bbc_content(row["url"])

        for model_name, sentiment_column in sentiment_models:
            sentiment_scores = []

            if content is not None:
                # Truncate or split the content into smaller chunks
                max_sequence_length = 512
                if len(content) > max_sequence_length:
                    content_chunks = [content[i:i+max_sequence_length] for i in range(0, len(content), max_sequence_length)]
                else:
                    content_chunks = [content]

                # Assess sentiment for each content chunk
                for chunk in content_chunks:
                    sentiment, score = assess_sentiment_model(chunk, model_name)
                    sentiment_scores.append(score)

            # Calculate average sentiment score
            if sentiment_scores:
                average_score = sum(sentiment_scores) / len(sentiment_scores)
                df.at[index, sentiment_column + "_average_score"] = average_score

    return df


In [3]:
df17 = pd.read_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2017_seperate_assessed_3.csv')
df18= pd.read_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2018_seperate_assessed_3.csv')



In [4]:
df_17 = assess_sentiment(df17)
df_17.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2017_seperate_assessed_5.csv', index=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a B

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

Streaming output truncated to the last 5000 lines.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down

In [5]:
df_18 = assess_sentiment(df18)
df_18.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2018_seperate_assessed_5.csv', index=True)

Streaming output truncated to the last 5000 lines.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down

In [9]:
df_18

,Unnamed: 0,title,date,url,player,textblob_Sentiment,key_words_score,sentiment_bert_average_score,sentiment_roberta_average_score,sentiment_xlnet_average_score,sentiment_distilbert_average_score
0,124,LeBron James impresses but LA Lakers lose 128-...,2018-19,https://www.bbc.co.uk/sport/basketball/45911389,LeBron James,0.162222,1.2,0.629764,0.504845,0.539852,0.523432
1,126,NBA: LeBron James claims first Los Angeles Lak...,2018-19,https://www.bbc.co.uk/sport/basketball/45975260,LeBron James,0.081929,1.2,0.65815,0.53622,0.576466,0.525603
2,128,LeBron James landmark as Los Angeles Lakers th...,2018-19,https://www.bbc.co.uk/sport/basketball/47141238,LeBron James,0.095989,1.0,0.708904,0.535416,0.589857,0.545908
3,129,LeBron James and the Los Angeles Lakers out of...,2018-19,https://www.bbc.co.uk/sport/basketball/47589985,LeBron James,0.102822,1.2,0.608637,0.542405,0.589194,0.515875
4,130,NBA: LeBron James overtakes Michael Jordan in ...,2018-19,https://www.bbc.co.uk/sport/basketball/47454448,LeBron James,0.090157,1.0,0.608305,0.538623,0.579211,0.522496
...,...,...,...,...,...,...,...,...,...,...,...
194,10564,NFL week 14 review: Chicago Bears channel 1985...,2018-19,https://www.bbc.co.uk/sport/american-football/...,Patrick Beverley,0.250917,0.6,0.609847,0.539803,0.584593,0.521131
195,10570,NFL week 13 review: Third time lucky as Charge...,2018-19,https://www.bbc.co.uk/sport/american-football/...,Patrick Beverley,0.112749,0.8,0.594471,0.529891,0.610279,0.52362
196,10579,NFL week seven review: Saints quarterback Bree...,2018-19,https://www.bbc.co.uk/sport/american-football/...,Patrick Beverley,0.099391,0.8,0.602186,0.538312,0.569233,0.522397
197,10612,NBA: LeBron James claims first Los Angeles Lak...,2018-19,https://www.bbc.co.uk/sport/basketball/45975260,Andre Ingram,0.081929,1.0,0.591267,0.515282,0.611849,0.526089


In [8]:
df_18.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2018_seperate_assessed_5.csv', index=False)
df_17.to_csv('/content/drive/MyDrive/DS105L/bbc_news_season/bbc_2017_seperate_assessed_5.csv', index=False)